In [81]:
import torch
from torch import nn
import numpy as np
import pandas as pd

In [82]:
df = pd.read_csv('data/train.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
X = torch.Tensor(df.values[:,1:]).to('cuda')
X = X.reshape(-1,28,28)
Y = torch.Tensor(df.values[:,0]).to('cuda')
print(X.shape, Y.shape)

torch.Size([42000, 28, 28]) torch.Size([42000])


In [84]:
from torch.utils.data import TensorDataset, DataLoader

In [85]:
dataset = TensorDataset(X,Y)

In [86]:
data_loader = DataLoader(dataset,50,shuffle=True,drop_last=True)

In [87]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,padding=1), # 28*28*32
            nn.ReLU(),
            nn.MaxPool2d(2) # 14*14*32
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3,padding=1), # 14*14*64
            nn.ReLU(),
            nn.MaxPool2d(2,padding=1) # 8*8*64
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1), # 8 * 8 * 128
            nn.ReLU(),
            nn.MaxPool2d(2) # 4 * 4 * 128
        )
        
        self.fc = nn.Linear(4*4*128,10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.fc(out)
        return out
        

In [88]:
model = CNN().to('cuda')
optimizer = torch.optim.Adam(model.parameters(),lr = 0.05)
criterion = torch.nn.CrossEntropyLoss().to('cuda')

In [89]:
len(data_loader)

840

In [90]:
for epoch in range(100):
    avg_cost = 0

    for X,Y in data_loader :
        X = X.to('cuda')
        Y = Y.to('cuda')
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / 840
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 50, 28, 28] to have 1 channels, but got 50 channels instead